# Creative Testing Significance Testing

Use cases:
What is accounted for and not accounted for by this tool
Format that the data has to be in

#### Import libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest as ztest

from authentication.authenticator import Authenticator
from sheets.sheetmanager import SheetManager

#### Options Menu

csv_in: True or False. True to read in a csv. False to read in a gsheet.

csv_out: True or False. True to output results to a csv. False to output results to a gsheet.

If either is False, gsheet spreadsheetId and data range must be provided. Keys (json) and scopes must be provided.

In [2]:
csv_in = False
csv_out = False

results_dp = 2

#### Initialize necessary strings for gsheets
Necessary strings if either csv_in or csv_out is False.

In [2]:
keys = 'credentials.json'
SCOPES = ['https://www.googleapis.com/auth/drive']

data_spreadsheetId = '1Yr-jmDKJixCcRRBK73yaWU-VtxZlcW5bnwm8wHqm538'
data_data_range = 'Data'

results_spreadsheetId = '1obf3qgInjFVS3F4IoLRMDItwOxT_CvOFsecP5WGVFr4'
results_data_range = 'Tests'

#### Initialize necessary strings for csv
Necessary strings if either csv_in or csv_out is True.

In [ ]:
# csv_path = 'test.csv'
# csv_out_path = 'results.csv'

#### Authenticate and Initialize Manager to work with Google Sheets

In [3]:
if (not csv_in) or (not csv_out):
    authenticator = Authenticator(keys)
    creds = authenticator.get_creds(SCOPES)
    manager = SheetManager(creds)

#### Load in Raw Data as DataFrame

In [4]:
if not csv_in:
    data_df = manager.get_values(spreadsheetId=data_spreadsheetId,
                            data_range=data_data_range)
else:
    data_df = pd.read_csv(csv_path)

Define questions here:
1. The Grouping question, and define the control
2. The weights question / variable. If no weights (like some Metrix Lab studies), leave as None


In [1]:
group_question = 'Q140'
control_value = '0'

weights_variable_name = 'Default Weights' # or None

In [ ]:
if weights_variable_name is None:
    weights_variable_name = 'Count'
    data_df[weights_variable_name] = 1

#### Recoding
Recode questions

1. Indicate demo questions so we won't pivot them but yet keep them around for cuts
2. Create dictionary of questions we want tested

In [6]:
######
# demo questions
######
demo_question_list = [
    'Q121',
    'Q122',
    'Q128',
    'Q129',
    'Q130',
    'Q131',
]

### Follow the format of
"""
{
 'question_id': [desired option 1, desired option 2(, desired option 3, ...)],
 'question_id': [desired option 1, desired option 2(, desired option 3, ...)]
}
"""

### TODO: take into account "Not Shown" answer options (N/A) answer options

desired_dic = {
    'Q132.1': [2],
    'Q132.2': [2],
    'Q132.3': [2],
    'Q132.4': [2],
    'Q132.5': [2],
    'Q132.6': [2],
    'Q132.7': [2],
    'Q132.8': [2],
    'Q132.9': [2],
    'Q133.1': [2],
    'Q133.2': [2],
    'Q133.3': [2],
    'Q133.4': [2],
    'Q133.5': [2],
    'Q133.6': [2],
    'Q133.7': [2],
    'Q133.8': [2],
    'Q133.9': [2],
    'Q133.10': [2],
    'Q161': [0, 1],
    'Q162.1': [2],
    'Q162.2': [2],
    'Q162.3': [2],
    'Q162.4': [2],
    'Q162.5': [2],
    'Q162.6': [2],
    'Q162.7': [2],
    'Q162.8': [2],
    'Q162.9': [2],
    'Q162.10': [2],
    'Q163.1': [2],
    'Q163.2': [2],
    'Q163.3': [2],
    'Q163.4': [2],
    'Q163.5': [2],
    'Q163.6': [2],
    'Q163.7': [2],
    'Q163.8': [2],
    'Q163.9': [2],
    'Q163.10': [2],
    'Q164.1': [2],
    'Q164.2': [2],
    'Q164.3': [2],
    'Q164.4': [2],
    'Q164.5': [2],
    'Q164.6': [2],
    'Q164.7': [2],
    'Q164.8': [2],
    'Q165.1': [2],
    'Q165.2': [2],
    'Q165.3': [2],
    'Q165.4': [2],
    'Q165.5': [2],
    'Q165.6': [2],
    'Q165.7': [2],
    'Q165.8': [2],
    
}


3. Recode the values indicated above to 1 (desired) or 0 (not desired)

In [7]:
def recode_desired(series):
    series = series.where(series.isin(desired_dic[series.name]), 'X')
    series = series.mask(series.isin(desired_dic[series.name]), 1)
    series[series=='X'] = 0
    series = pd.to_numeric(series)
    return(series)

test_questions = list(desired_dic.keys())

recoded_df = data_df.copy()
recoded_df[test_questions] = recoded_df[test_questions].apply(recode_desired, axis=0)
recoded_df

,Default Weights,Q121,Q122,Q128,Q129,Q130,Q131,Q132.1,Q132.2,Q132.3,...,Q186,Q187,Q188,Q189,Q190,Q191,Q192,Q193,Q194,Q195
0,0.771605,15,1,0,2,0,0,1,0,0,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.488095,18,0,1,3,1,2,0,0,0,...,2,分からない,ない,1.0,1.0,1.0,1.0,0.0,1.0,1.0
2,0.744048,17,1,4,0,1,2,0,1,1,...,1,なりたい自分になる,具体的なことがわからない,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.077586,25,1,7,1,0,0,0,0,1,...,2,料理,落ち着かない,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.644737,16,0,2,1,1,0,0,0,0,...,2,特にない,特にない,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.358696,25,1,4,2,1,3,1,0,0,...,2,飯テロ動画のYouTuberの紹介,嫌いではないが、見ていてお腹がすきます,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.059322,27,0,4,0,1,2,0,1,0,...,2,メイクの方法。,メイクの、クローズアップ,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,0.905797,21,1,2,2,0,3,1,0,1,...,2,わかりません。,ユーチューバー,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,0.905797,21,1,3,2,0,0,0,0,1,...,1,特になし,嫌いな点はない,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,0.672043,20,1,2,2,1,1,1,1,1,...,1,器具を使わずにトレーニングできる,なし,1.0,1.0,1.0,1.0,1.0,1.0,1.0


#### Cuts
1. Identify the cuts you want, and save them as Data Frames.
2. Then, create a dictionary of cut names and their corresponding Data Frames

In [8]:
overall_df = recoded_df.copy()

male = overall_df[overall_df['Q122'] == 0]
female = overall_df[overall_df['Q122'] == 1]
casual = overall_df[(overall_df['Q131'] == 1) | (overall_df['Q131'] == 4)]

cuts_dic = {
    'Overall': overall_df,
    'Male': male,
    'Female': female,
    'Casual': casual
}

#### Execute Tests

Execute the tests on all the cuts indicated above, for the questions that have been selected for testing.

TODO: Specify DP for columns

In [14]:
tables = []
control_value = str(control_value)
for cut in cuts_dic.keys():    
    table = cuts_dic[cut].copy()
    table[group_question] = table[group_question].astype(str)
    weights_table = table[[group_question, weights_variable_name]]
    bases = weights_table.groupby(table[group_question]).sum()[weights_variable_name]
    questions = test_questions
    table = pd.pivot_table(table, values=questions, columns=group_question, aggfunc='sum')
    initial_cols = table.columns
    for group in table.columns:
        table[group+' Base'] = bases[group]
        table[group+' Desired %'] = table[group] / table[group+' Base']
        if group != control_value:
            table[group+' Abs Lift %'] = table[group+' Desired %'] - table[control_value+' Desired %']
            table[group+' p-value'] = table.apply(lambda x: ztest(
                                                                [x[group], x[control_value]],
                                                                [x[group+' Base'], x[control_value+' Base']]
                                                                    )[1], axis=1)
            table[group+' Significance'] = 'No Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .2) & (table[group+' Abs Lift %'] > 0)] = 'Directional Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .1) & (table[group+' Abs Lift %'] > 0)] = 'Significant Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .05) & (table[group+' Abs Lift %'] > 0)] = 'Strong Significant Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .1) & (table[group+' Abs Lift %'] < 0)] = 'Negative Lift'

    suffixes = [' Base', ' Desired %', ' Abs Lift %', ' Significance'] # can include control_name+' p-value'
    table['Cut'] = cut
    col_order = ['Cut', control_value, control_value+' Base', control_value+' Desired %']
    for col in initial_cols:
        if col != control_value:
            col_order.append(col)
            for suffix in suffixes:
                col_order.append(col+suffix)
    table = table[col_order]
    #table = table.rename(index=codebook_dic)
    tables.append(table)
    
results = pd.concat(tables, axis=0)

C:\Users\charles.tan\Anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:670: RuntimeWarning: invalid value encountered in double_scalars
  zstat = value / std_diff
C:\Users\charles.tan\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\charles.tan\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\charles.tan\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\charles.tan\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

Write results automatically to gsheets, or output a csv. Based on selected option.

In [15]:
if not csv_out:
    results_to_write = results.reset_index()

    values = []
    values.append(results_to_write.columns.tolist())
    for row in results_to_write.values.tolist():
        values.append(row)

    if results_data_range not in manager.get_existing_sheets_names(spreadsheetId=results_spreadsheetId):
        manager.add_sheet(spreadsheetId=results_spreadsheetId, sheet_name=results_data_range)

    manager.update_values(spreadsheetId=results_spreadsheetId,
                          update_range=results_data_range,
                          values=values)
else:
    results.to_csv(index=True, header=True)

{'spreadsheetId': '1obf3qgInjFVS3F4IoLRMDItwOxT_CvOFsecP5WGVFr4',
 'updatedRange': 'Tests!A1:AX225',
 'updatedRows': 225,
 'updatedColumns': 50,
 'updatedCells': 11250}